In [69]:
import time
import pandas as pd
import datetime as dt

_dt_dt_ = dt.datetime
_dt_d_  = dt.date

from binance.client import Client

In [15]:
api_key_info = pd.read_csv('/home/yizhang/Research/api_key.csv')
api_key      = api_key_info.loc[0, 'value']
api_secret   = api_key_info.loc[1, 'value']

In [16]:
client = Client(api_key, api_secret)

In [112]:
def clean_trade_data(trades):
    trades[['price', 'qty', 'time', 'bid', 'ask']] = trades[['price', 'qty', 'time', 'bid', 'ask']].astype(float)
    trades.loc[:, 'sign'] = 0
    trades.loc[:, 'mid']  = 0.5 * (trades.bid + trades.ask)
    trades.loc[trades.price < trades.mid, 'sign'] = -1
    trades.loc[trades.price > trades.mid, 'sign'] = 1 

In [126]:
# get market depth
symbol    = 'ETHBTC'
delay     = 1
iteration = 0

MAX_ITERATIONS = 10

while(iteration < MAX_ITERATIONS):
    iteration += 1
    
    depth   = pd.DataFrame(client.get_order_book(symbol=symbol))
    
    bestAsk = depth.loc[0, 'asks']
    bestBid = depth.loc[0, 'bids']
    
    trades  = pd.DataFrame(client.get_recent_trades(symbol=symbol))
    trades.loc[:, 'bid'] = bestBid[0]
    trades.loc[:, 'ask'] = bestAsk[0]
    
    clean_trade_data(trades)
    
    print ("%s: %s,%s -- %s,%s" % (_dt_dt_.now().strftime("%Y%m%d %H:%M:%S.%f"), bestAsk[1], bestAsk[0], bestBid[0], bestBid[1]))
    time.sleep(delay)

20180603 15:28:25.634418: 22.15600000,0.07833100 -- 0.07831100,49.99000000
20180603 15:28:26.861718: 19.05800000,0.07833100 -- 0.07831000,3.07900000
20180603 15:28:28.221690: 18.96900000,0.07833100 -- 0.07831000,0.00100000
20180603 15:28:29.426869: 18.96900000,0.07833100 -- 0.07830900,1.01400000
20180603 15:28:30.637458: 18.84000000,0.07833100 -- 0.07830900,0.95400000
20180603 15:28:31.853039: 18.84000000,0.07833100 -- 0.07830900,0.89400000
20180603 15:28:34.633423: 20.55500000,0.07833100 -- 0.07830900,0.72100000
20180603 15:28:36.028583: 20.13600000,0.07833100 -- 0.07830800,0.10000000
20180603 15:28:38.120978: 20.08600000,0.07833100 -- 0.07828400,5.44800000
20180603 15:28:39.943097: 19.95700000,0.07833100 -- 0.07828500,6.88600000
